# Data Preparation

In [8]:
#import necessary modules
import pandas as pd
import numpy as np

#read excel files obtained from EXIST from 2016-01-01 to 2024-12-10
list1 = []

for i in range(1, 5):
    df_1 = pd.read_excel("Gercek_Zamanli_Tuketim-"+str(i)+".xlsx")
    df_1['Tarih'] = pd.to_datetime(df_1['Tarih']).dt.date
    list1.append(df_1)

#merge files
df_latest = pd.concat(list1, ignore_index=True)
df_1 = df_latest.drop_duplicates()
df_1 = df_1.iloc[:,:3]
df_1.to_csv("latest_1.csv")
df_1


/opt/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Tarih,Saat,Tüketim Miktarı(MWh)
0,2016-01-01,00:00,26277.24
1,2016-01-01,01:00,24991.82
2,2016-01-01,02:00,23532.61
3,2016-01-01,03:00,22464.78
4,2016-01-01,04:00,22002.91
...,...,...,...
78403,2024-12-10,19:00,43827.32
78404,2024-12-10,20:00,42815.12
78405,2024-12-10,21:00,41780.45
78406,2024-12-10,22:00,40340.09


In [85]:
#determine the day and whether it is weekend using Tarih (date) column
import datetime

def date_to_day(date):
    weekday_index = date.weekday()
    
    weekday_names = ['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']
    
    return weekday_names[weekday_index]

df_1.loc[:,'Gun'] = df_1['Tarih'].apply(date_to_day)
df_1.loc[:,'HAFTASONU'] = (df_1['GUN'] == 'Pazar') | (df_1['GUN'] == 'Cumartesi')
df_1

,Tarih,Saat,Tüketim Miktarı(MWh),GUN,HAFTASONU
0,2016-01-01,00:00,26277.24,Cuma,False
1,2016-01-01,01:00,24991.82,Cuma,False
2,2016-01-01,02:00,23532.61,Cuma,False
3,2016-01-01,03:00,22464.78,Cuma,False
4,2016-01-01,04:00,22002.91,Cuma,False
...,...,...,...,...,...
78403,2024-12-10,19:00,43827.32,Salı,False
78404,2024-12-10,20:00,42815.12,Salı,False
78405,2024-12-10,21:00,41780.45,Salı,False
78406,2024-12-10,22:00,40340.09,Salı,False


In [87]:
#build holiday feature which is based on national holidays of Türkiye added as TATIL (holiday)

date = ['01-01', '04-23', '05-01', '05-19','07-15', '08-30', '10-29']
dini_bayram = [
    "2016-07-05", "2016-07-06", "2016-07-07", "2016-09-12", "2016-09-13", "2016-09-14", "2016-09-15",
    "2017-06-25", "2017-06-26", "2017-06-27", "2017-09-01", "2017-09-02", "2017-09-03", "2017-09-04",
    "2018-06-15", "2018-06-16", "2018-06-17", "2018-08-21", "2018-08-22", "2018-08-23", "2018-08-24",
    "2019-06-04", "2019-06-05", "2019-06-06", "2019-08-11", "2019-08-12", "2019-08-13", "2019-08-14",
    "2020-05-24", "2020-05-25", "2020-05-26", "2020-07-31", "2020-08-01", "2020-08-02", "2020-08-03",
    "2021-05-13", "2021-05-14", "2021-05-15", "2021-07-20", "2021-07-21", "2021-07-22", "2021-07-23",
    "2022-05-02", "2022-05-03", "2022-05-04", "2022-07-09", "2022-07-10", "2022-07-11", "2022-07-12",
    "2023-04-21", "2023-04-22", "2023-04-23", "2023-06-28", "2023-06-29", "2023-06-30", "2023-07-01",
    "2024-04-10", "2024-04-11", "2024-04-12","2024-06-16","2024-06-17","2024-06-18","2024-06-19"
]
milli_bayram=[]

for i in range(2016,2025):
    for j in date:
        i = str(i)
        milli_bayram.append(i+"-"+str(j))

milli_bayram.remove('2016-07-15')
df_1['Tarih'] = pd.to_datetime(df_1['Tarih'], format='%Y-%m-%d')

df_1['BAYRAM'] = df_1['Tarih'].isin(pd.to_datetime(dini_bayram)) | df_1['Tarih'].isin(pd.to_datetime(milli_bayram))
is_weekend = df_1['HAFTASONU']
is_holiday =df_1['BAYRAM'] 
df_1['TATIL'] = is_weekend | is_holiday
df_1

,Tarih,Saat,Tüketim Miktarı(MWh),GUN,HAFTASONU,BAYRAM,TATIL
0,2016-01-01,00:00,26277.24,Cuma,False,True,True
1,2016-01-01,01:00,24991.82,Cuma,False,True,True
2,2016-01-01,02:00,23532.61,Cuma,False,True,True
3,2016-01-01,03:00,22464.78,Cuma,False,True,True
4,2016-01-01,04:00,22002.91,Cuma,False,True,True
...,...,...,...,...,...,...,...
78403,2024-12-10,19:00,43827.32,Salı,False,False,False
78404,2024-12-10,20:00,42815.12,Salı,False,False,False
78405,2024-12-10,21:00,41780.45,Salı,False,False,False
78406,2024-12-10,22:00,40340.09,Salı,False,False,False


In [15]:
#fetch hourly temperature data using coordinates and meteostat API and assign to column with city's own name

from meteostat import Point, Hourly

start = datetime.datetime(2016, 1, 1, 1, 0)
end = datetime.datetime(2024, 12, 10, 0, 0)

turkey_cities = {
    "Adana": (37.0000, 35.3213, 23),
    "Adıyaman": (37.7648, 38.2786, 669),
    "Afyonkarahisar": (38.7507, 30.5566, 1028),
    "Ağrı": (39.7191, 43.0503, 1640),
    "Aksaray": (38.3787, 34.0270, 976),
    "Amasya": (40.6539, 35.8332, 411),
    "Ankara": (39.9334, 32.8597, 938),
    "Antalya": (36.8969, 30.7133, 30),
    "Ardahan": (41.1105, 42.7022, 1820),
    "Artvin": (41.1828, 41.8194, 628),
    "Aydın": (37.8444, 27.8456, 65),
    "Balıkesir": (39.6492, 27.8861, 120),
    "Bartın": (41.5811, 32.4598, 25),
    "Batman": (37.8812, 41.1351, 540),
    "Bayburt": (40.2552, 40.2249, 1555),
    "Bilecik": (40.1431, 29.9793, 500),
    "Bingöl": (38.8853, 40.4983, 1159),
    "Bitlis": (38.3938, 42.1230, 1550),
    "Bolu": (40.7350, 31.6069, 742),
    "Burdur": (37.7203, 30.2908, 950),
    "Bursa": (40.1828, 29.0665, 155),
    "Çanakkale": (40.1456, 26.4064, 5),
    "Çankırı": (40.5997, 33.6134, 739),
    "Çorum": (40.5489, 34.9535, 801),
    "Denizli": (37.7765, 29.0864, 354),
    "Diyarbakır": (37.9144, 40.2306, 660),
    "Düzce": (40.8438, 31.1565, 146),
    "Edirne": (41.6772, 26.5557, 41),
    "Elazığ": (38.6743, 39.2220, 1067),
    "Erzincan": (39.7477, 39.4911, 1185),
    "Erzurum": (39.9043, 41.2679, 1890),
    "Eskişehir": (39.7767, 30.5206, 788),
    "Gaziantep": (37.0662, 37.3833, 850),
    "Giresun": (40.9128, 38.3895, 40),
    "Gümüşhane": (40.4591, 39.4828, 1220),
    "Hakkari": (37.5830, 43.7339, 1720),
    "Hatay": (36.4018, 36.3498, 80),
    "Iğdır": (39.9197, 44.0450, 850),
    "Isparta": (37.7648, 30.5566, 1050),
    "İstanbul": (41.0082, 28.9784, 39),
    "İzmir": (38.4192, 27.1287, 10),
    "Kahramanmaraş": (37.5858, 36.9371, 568),
    "Karabük": (41.2061, 32.6204, 273),
    "Karaman": (37.1811, 33.2150, 1033),
    "Kars": (40.6025, 43.0970, 1768),
    "Kastamonu": (41.3887, 33.7827, 775),
    "Kayseri": (38.7225, 35.4875, 1050),
    "Kırıkkale": (39.8468, 33.5153, 740),
    "Kırklareli": (41.7351, 27.2253, 203),
    "Kırşehir": (39.1451, 34.1630, 980),
    "Kilis": (36.7165, 37.1150, 680),
    "Kocaeli": (40.7654, 29.9406, 1),
    "Konya": (37.8715, 32.4846, 1016),
    "Kütahya": (39.4221, 29.9833, 969),
    "Malatya": (38.3552, 38.3095, 964),
    "Manisa": (38.6191, 27.4289, 69),
    "Mardin": (37.3212, 40.7245, 1083),
    "Mersin": (36.8121, 34.6415, 10),
    "Muğla": (37.2153, 28.3636, 670),
    "Muş": (38.9462, 41.7539, 1350),
    "Nevşehir": (38.6240, 34.7239, 1220),
    "Niğde": (37.9667, 34.6795, 1290),
    "Ordu": (40.9863, 37.8785, 4),
    "Osmaniye": (37.0742, 36.2470, 118),
    "Rize": (41.0201, 40.5234, 9),
    "Sakarya": (40.7731, 30.3957, 31),
    "Samsun": (41.2867, 36.33, 10),
    "Siirt": (37.9333, 41.9467, 902),
    "Sinop": (42.0231, 35.1531, 30),
    "Sivas": (39.7477, 37.0179, 1285),
    "Şanlıurfa": (37.1676, 38.7955, 518),
    "Şırnak": (37.5161, 42.4621, 1360),
    "Tekirdağ": (40.9780, 27.5119, 4),
    "Tokat": (40.3169, 36.5544, 623),
    "Trabzon": (41.005, 39.7269, 5),
    "Tunceli": (39.1081, 39.5459, 940),
    "Uşak": (38.6823, 29.4082, 906),
    "Van": (38.5012, 43.4021, 1720),
    "Yalova": (40.6500, 29.2742, 30),
    "Yozgat": (39.8181, 34.8147, 1300),
    "Zonguldak": (41.4564, 31.7987, 63)
}

index = pd.date_range(start=start, end=end, freq='H')
df_temp_data = pd.DataFrame(index=index)

#If no data available, assign the average of 3 neighboring cities' temperature

for city, (lat, lon, elevation) in turkey_cities.items():
    point = Point(lat, lon, elevation)
    data = Hourly(point, start, end)
    data = data.fetch()
    
    if 'temp' in data.columns and not data['temp'].empty:
        data = data.reindex(index, fill_value=None)
        df_temp_data[city] = data['temp'].values
    else:
        closest_cities_data = []
        min_distances = [float('inf')] * 3
        closest_cities = [None] * 3
        
        for neighbor_city, (neighbor_lat, neighbor_lon, _) in turkey_cities.items():
            if neighbor_city != city:
                distance = (lat - neighbor_lat)**2 + (lon - neighbor_lon)**2
                for i in range(3):
                    if distance < min_distances[i]:
                        min_distances.insert(i, distance)
                        closest_cities.insert(i, neighbor_city)
                        min_distances.pop()
                        closest_cities.pop()
                        break

        for i in range(3):
            if closest_cities[i] in df_temp_data.columns:
                closest_cities_data.append(np.array(df_temp_data[closest_cities[i]].values, dtype=np.float64))
            else:
                closest_cities_data.append(np.full(len(index), np.nan))

        closest_cities_data = np.array(closest_cities_data, dtype=np.float64)
        avg_temp = np.nanmean(closest_cities_data, axis=0)
        df_temp_data[city] = avg_temp

print(df_temp_data)


                     Adana  Adıyaman  Afyonkarahisar  Ağrı  Aksaray  Amasya  \
2016-01-01 01:00:00    5.0       NaN            -9.3   NaN      NaN    -7.2   
2016-01-01 02:00:00    4.0       NaN            -9.5   NaN      NaN    -8.2   
2016-01-01 03:00:00    4.0       0.2            -9.7  -9.2      NaN    -8.5   
2016-01-01 04:00:00    4.0      -1.0           -10.0   NaN      NaN    -8.2   
2016-01-01 05:00:00    4.0      -1.0            -9.8   NaN      NaN    -9.2   
...                    ...       ...             ...   ...      ...     ...   
2024-12-09 20:00:00   11.2       5.3             6.9  -3.0      NaN     5.8   
2024-12-09 21:00:00   10.8       5.4             6.4  -3.2      NaN     5.6   
2024-12-09 22:00:00   10.6       5.6             6.2  -3.5      NaN     5.4   
2024-12-09 23:00:00   10.4       5.7             5.9  -3.9      NaN     5.2   
2024-12-10 00:00:00    9.9       5.5             5.6  -4.3      NaN     4.9   

                     Ankara  Antalya  Ardahan  Artv

In [65]:
#save temperature data to be merged later

df_temp_data.to_csv("son_temp.csv")

In [121]:
#count total number of null values in the DataFrame

total_null_values = b.isnull().sum().sum()
print(f"Total number of null values in the DataFrame: {total_null_values}")

Total number of null values in the DataFrame: 82


In [61]:
#There are still missing values, which shows that there are cities which doesn't have neighboring city data

#identify columns with all null values in the first 43861 rows which is the first big empty space
a = df_temp_data.iloc[43862:, :]
null_columns = a.columns[a.isnull().all()]

#this time, find up to 5 closest cities with available temperature data
for city in null_columns:
    lat, lon, _ = turkey_cities[city]
    
    closest_cities = [None] * 5
    min_distances = [float('inf')] * 5
    

    for neighbor_city, (neighbor_lat, neighbor_lon, _) in turkey_cities.items():
        if neighbor_city != city and neighbor_city in df_temp_data.columns:
            distance = (lat - neighbor_lat) ** 2 + (lon - neighbor_lon) ** 2
            for i in range(5):
                if distance < min_distances[i]:
                    min_distances.insert(i, distance)
                    closest_cities.insert(i, neighbor_city)
                    min_distances.pop()
                    closest_cities.pop()
                    break
    
#check if any suitable neighbor city was found with valid data
    
    for i, neighbor_city in enumerate(closest_cities):
        if neighbor_city is not None and not pd.isna(df_temp_data.loc[df_temp_data.index[43862:], neighbor_city].values).all():
            df_temp_data.loc[df_temp_data.index[43862:], city] = df_temp_data.loc[df_temp_data.index[43862:], neighbor_city]
            print(f"Filling {city} with data from closest neighbor: {neighbor_city} (Neighbor {i + 1})")
            break
    else:
        print(f"No suitable neighbor found for {city}. Filling with None values.")
        df_temp_data.loc[df_temp_data.index[43862:], city] = None

#merge filled data with existing df_temp_data 
df_temp_data = pd.concat([a, df_temp_data.loc[df_temp_data.index[:43862], :]], axis=0)

#show updated data
print(df_temp_data)

Filling Aksaray with data from closest neighbor: Nevşehir (Neighbor 1)
Filling Bartın with data from closest neighbor: Karabük (Neighbor 1)
Filling Bayburt with data from closest neighbor: Gümüşhane (Neighbor 1)
Filling Bilecik with data from closest neighbor: Kocaeli (Neighbor 1)
Filling Bitlis with data from closest neighbor: Siirt (Neighbor 1)
Filling Hakkari with data from closest neighbor: Van (Neighbor 1)
Filling Karaman with data from closest neighbor: Konya (Neighbor 1)
                     Adana  Adıyaman  Afyonkarahisar  Ağrı  Aksaray  Amasya  \
2021-01-01 15:00:00   17.6      10.3            10.7  10.0      3.2     9.2   
2021-01-01 16:00:00   16.0       9.8             9.2  11.0      3.4     6.8   
2021-01-01 17:00:00   15.0       9.8             8.1  12.0      4.2     4.8   
2021-01-01 18:00:00   14.6       9.5             7.6  10.0      3.0     5.8   
2021-01-01 19:00:00   15.0       8.8             7.0   9.0      2.5     3.8   
...                    ...       ...       

In [91]:
#combining consumption data with temperature data

df2=pd.read_csv("son_temp.csv")
df_final = pd.concat([df_1, df2], axis=1)
print(df_final)

           Tarih   Saat  Tüketim Miktarı(MWh)   GUN HAFTASONU BAYRAM  TATIL  \
0     2016-01-01  00:00              26277.24  Cuma     False   True   True   
1     2016-01-01  01:00              24991.82  Cuma     False   True   True   
2     2016-01-01  02:00              23532.61  Cuma     False   True   True   
3     2016-01-01  03:00              22464.78  Cuma     False   True   True   
4     2016-01-01  04:00              22002.91  Cuma     False   True   True   
...          ...    ...                   ...   ...       ...    ...    ...   
78404 2024-12-10  20:00              42815.12  Salı     False  False  False   
78405 2024-12-10  21:00              41780.45  Salı     False  False  False   
78406 2024-12-10  22:00              40340.09  Salı     False  False  False   
78407 2024-12-10  23:00              37931.39  Salı     False  False  False   
2068         NaT    NaN                   NaN   NaN       NaN    NaN    NaN   

                Unnamed: 0  Adana  Adıyaman  ...  Ş

In [125]:
#slice available up-to-date data only from the end

df_final= df_final.iloc[:78383,:]
df_final.to_csv("latest_2.csv")

In [145]:
#read population data from TURKSTAT excel file 

pop = pd.read_excel("pop_latest.xlsx")

pop=pop.drop(columns=["Unnamed: 0"],axis=1)
pop.columns=["Year","City","Population"]
pop
pop.to_excel("pop_latest.xlsx")

In [151]:
#read population data of the cities between 2016 and 2024. Saving as "city_name_pop" 

city=pd.Series(pop["City"])
df=pd.read_csv("latest_2.csv")
new_df=pd.DataFrame()
for i in city:
    name=i+"_pop"
    for a in range(2016,2024):
        filtered_data = pop[(pop['Year'] == a) & (pop['City'] == i)]
        new_df.loc[a, name] = filtered_data["Population"].values[0]
new_df.to_excel("pop_years.xlsx")

new_df.index = range(1,9)
new_df["Year"]=range(2016,2024)
df["Tarih"] = pd.to_datetime(df["Tarih"])
df["Year"] = df["Tarih"].dt.year
df

,Unnamed: 0.1,Tarih,Saat,Tüketim Miktarı(MWh),GUN,HAFTASONU,BAYRAM,TATIL,Unnamed: 0,Adana,...,Tekirdağ,Tokat,Trabzon,Tunceli,Uşak,Van,Yalova,Yozgat,Zonguldak,Year
0,0,2016-01-01,00:00,26277.24,Cuma,False,True,True,2021-01-01 15:00:00,17.6,...,13.6,9.4,13.7,2.233333,7.6,-3.3,13.5,0.8,13.6,2016
1,1,2016-01-01,01:00,24991.82,Cuma,False,True,True,2021-01-01 16:00:00,16.0,...,11.9,6.6,14.2,0.966667,7.3,-3.3,12.2,1.1,13.2,2016
2,2,2016-01-01,02:00,23532.61,Cuma,False,True,True,2021-01-01 17:00:00,15.0,...,12.3,6.3,14.2,0.533333,7.0,-5.3,11.8,1.3,12.4,2016
3,3,2016-01-01,03:00,22464.78,Cuma,False,True,True,2021-01-01 18:00:00,14.6,...,11.5,5.0,14.8,-0.766667,6.0,-5.5,10.8,1.9,13.0,2016
4,4,2016-01-01,04:00,22002.91,Cuma,False,True,True,2021-01-01 19:00:00,15.0,...,9.6,4.2,13.2,-0.633333,5.1,-6.3,10.4,1.4,13.0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78378,78379,2024-12-09,19:00,43380.10,Pazartesi,False,False,False,2021-01-01 10:00:00,20.0,...,12.9,8.8,17.2,3.900000,12.1,4.7,13.1,10.8,14.2,2024
78379,78380,2024-12-09,20:00,42118.38,Pazartesi,False,False,False,2021-01-01 11:00:00,21.0,...,13.0,10.3,16.2,4.900000,12.1,3.7,13.8,12.8,13.8,2024
78380,78381,2024-12-09,21:00,41075.61,Pazartesi,False,False,False,2021-01-01 12:00:00,21.2,...,14.1,13.0,17.2,7.000000,11.1,3.5,14.8,13.3,14.6,2024
78381,78382,2024-12-09,22:00,39607.45,Pazartesi,False,False,False,2021-01-01 13:00:00,21.0,...,16.9,13.2,17.2,6.900000,8.1,2.7,16.1,14.8,14.6,2024


In [153]:
#fix the index for pop df

pop.index= pop.Year
df.index=df.Year

In [157]:
#assign the population to the main df

for a in range(2016,2024):
    for i in pop["City"]:
        name=i+"_pop"
        if not df[df["Year"]==a].empty:
            temp=pop.loc[a,]
            
            df.loc[a,name]=temp[temp["City"]==i]["Population"]
        else:
            df[name]=None
df

,Unnamed: 0.1,Tarih,Saat,Tüketim Miktarı(MWh),GUN,HAFTASONU,BAYRAM,TATIL,Unnamed: 0,Adana,...,Şırnak_pop,Tekirdağ_pop,Tokat_pop,Trabzon_pop,Tunceli_pop,Uşak_pop,Van_pop,Yalova_pop,Yozgat_pop,Zonguldak_pop
Year,,,,,,,,,,,,,,,,,,,,,
2016,0,2016-01-01,00:00,26277.24,Cuma,False,True,True,2021-01-01 15:00:00,17.6,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,1,2016-01-01,01:00,24991.82,Cuma,False,True,True,2021-01-01 16:00:00,16.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,2,2016-01-01,02:00,23532.61,Cuma,False,True,True,2021-01-01 17:00:00,15.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,3,2016-01-01,03:00,22464.78,Cuma,False,True,True,2021-01-01 18:00:00,14.6,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,4,2016-01-01,04:00,22002.91,Cuma,False,True,True,2021-01-01 19:00:00,15.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,78379,2024-12-09,19:00,43380.10,Pazartesi,False,False,False,2021-01-01 10:00:00,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,78380,2024-12-09,20:00,42118.38,Pazartesi,False,False,False,2021-01-01 11:00:00,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,78381,2024-12-09,21:00,41075.61,Pazartesi,False,False,False,2021-01-01 12:00:00,21.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
#assign 2023 population data to 2024 in df if data don't exist recently

for index, row in pop.iterrows():
    city = row['City']
    population_2023 = row['Population'] if index == 2023 else None
    
    df.loc[df["Year"] == 2024, f"{city}_pop"] = population_2023
df

,Unnamed: 0.1,Tarih,Saat,Tüketim Miktarı(MWh),GUN,HAFTASONU,BAYRAM,TATIL,Unnamed: 0,Adana,...,Şırnak_pop,Tekirdağ_pop,Tokat_pop,Trabzon_pop,Tunceli_pop,Uşak_pop,Van_pop,Yalova_pop,Yozgat_pop,Zonguldak_pop
Year,,,,,,,,,,,,,,,,,,,,,
2016,0,2016-01-01,00:00,26277.24,Cuma,False,True,True,2021-01-01 15:00:00,17.6,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,1,2016-01-01,01:00,24991.82,Cuma,False,True,True,2021-01-01 16:00:00,16.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,2,2016-01-01,02:00,23532.61,Cuma,False,True,True,2021-01-01 17:00:00,15.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,3,2016-01-01,03:00,22464.78,Cuma,False,True,True,2021-01-01 18:00:00,14.6,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2016,4,2016-01-01,04:00,22002.91,Cuma,False,True,True,2021-01-01 19:00:00,15.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,78379,2024-12-09,19:00,43380.10,Pazartesi,False,False,False,2021-01-01 10:00:00,20.0,...,569751.0,1158621.0,599772.0,810537.0,88871.0,368963.0,1123997.0,279969.0,410875.0,586703.0
2024,78380,2024-12-09,20:00,42118.38,Pazartesi,False,False,False,2021-01-01 11:00:00,21.0,...,569751.0,1158621.0,599772.0,810537.0,88871.0,368963.0,1123997.0,279969.0,410875.0,586703.0
2024,78381,2024-12-09,21:00,41075.61,Pazartesi,False,False,False,2021-01-01 12:00:00,21.2,...,569751.0,1158621.0,599772.0,810537.0,88871.0,368963.0,1123997.0,279969.0,410875.0,586703.0


In [167]:
#reset the index and dropping column named Unnamed

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.reset_index(drop=True, inplace=True)
df.index.name = "Index"

#display the final version of the df
df

,Tarih,Saat,Tüketim Miktarı(MWh),GUN,HAFTASONU,BAYRAM,TATIL,Adana,Adıyaman,Afyonkarahisar,...,Şırnak_pop,Tekirdağ_pop,Tokat_pop,Trabzon_pop,Tunceli_pop,Uşak_pop,Van_pop,Yalova_pop,Yozgat_pop,Zonguldak_pop
Index,,,,,,,,,,,,,,,,,,,,,
0,2016-01-01,00:00,26277.24,Cuma,False,True,True,17.6,10.3,10.7,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
1,2016-01-01,01:00,24991.82,Cuma,False,True,True,16.0,9.8,9.2,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
2,2016-01-01,02:00,23532.61,Cuma,False,True,True,15.0,9.8,8.1,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
3,2016-01-01,03:00,22464.78,Cuma,False,True,True,14.6,9.5,7.6,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
4,2016-01-01,04:00,22002.91,Cuma,False,True,True,15.0,8.8,7.0,...,483125.0,969128.0,597994.0,773058.0,82053.0,351194.0,1097762.0,230538.0,413883.0,593718.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78378,2024-12-09,19:00,43380.10,Pazartesi,False,False,False,20.0,12.1,11.1,...,569751.0,1158621.0,599772.0,810537.0,88871.0,368963.0,1123997.0,279969.0,410875.0,586703.0
78379,2024-12-09,20:00,42118.38,Pazartesi,False,False,False,21.0,12.7,11.8,...,569751.0,1158621.0,599772.0,810537.0,88871.0,368963.0,1123997.0,279969.0,410875.0,586703.0
78380,2024-12-09,21:00,41075.61,Pazartesi,False,False,False,21.2,14.1,12.4,...,569751.0,1158621.0,599772.0,810537.0,88871.0,368963.0,1123997.0,279969.0,410875.0,586703.0


In [171]:
#save the df as most_latest.csv
df.to_csv("most_latest.csv")